# Сентимент-анализ текстов на русском языке

## Основные этапы
1. [Предобработка](#предобработка)  

2. [Машинное обучение](#машинное-обучение)  

    2.1 [Тестирование различных моделей c BOW и tf-idf](#тестирование-различных-моделей-c-bow-и-tf-idf)  
    - [Логистическая регрессия](#логистическая-регрессия)  
    - [Наивный байесовский классификатор](#наивный-байесовский-классификатор)  
    - [Метод K-ближайших соседей](#метод-k-ближайших-соседей)  
    - [Решающее дерево](#решающее-дерево)  
    - [Градиентный бустинг](#градиентный-бустинг)  

    2.2 [Тестирование различных моделей с word2vec](#тестирование-различных-моделей-с-word2vec)
    - [Логистическая регрессия](#логистическая-регрессия-1)
    - [Градиентный бустинг](#градиентный-бустинг-1)

Если надо запустить код, то стоит скачать папку data и положить в папку проекта  
Ссылка: https://drive.google.com/drive/folders/1A69dyu6-qFTtwLl0wiF3J7W6dF6zhJMi?usp=sharing

In [1]:
import os
import logging
from tqdm.notebook import tqdm
from joblib import dump, load

import pandas as pd
import numpy as np

import nltk
from pymorphy3 import MorphAnalyzer
from nltk.corpus import stopwords
import re

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import Word2Vec

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner
from sklearn.model_selection import cross_val_score
from sklearn.utils.class_weight import compute_class_weight

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

Запуск скрипта для загрузки тренировочных и тестовых данных (можно не комментировать, в случае 
если данные уже загружены - это учитывается) и загрузка данных в датафреймы

In [2]:
current_dir = os.getcwd()
project_dir = os.path.abspath(os.path.join(current_dir, '..'))

data_load_script_path = os.path.join(project_dir, "scripts\\data_load.py")
!python {data_load_script_path}

data_dir = os.path.join(project_dir, 'data')

train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
test = pd.read_csv(os.path.join(data_dir, 'test.csv'))

Посмотрим на данные

In [3]:
train.head(5)

,Unnamed: 0,text,sentiment
0,21098,".с.,и спросил его: о Посланник Аллаха!Ты пори...",1
1,21099,Роднее всех родных Попала я в ГКБ №8 еще в дек...,1
2,21100,Непорядочное отношение к своим работникам Рабо...,2
3,21101,"). Отсутствуют нормативы, Госты и прочее, что ...",1
4,21102,У меня машина в руках 5 лет и это п...,1


0 - нейтральный, 1 - позитивный, 2 - негативный.  
Можно отбросить признак Unnamed: 0

In [4]:
train.drop(columns=["Unnamed: 0"], inplace=True)
test.drop(columns=["Unnamed: 0"], inplace=True)

In [5]:
train.describe()

,sentiment
count,189891.00000
mean,1.00248
std,0.72250
min,0.00000
25%,0.00000
50%,1.00000
75%,2.00000
max,2.00000


In [6]:
train.nunique()

text         181103
sentiment         3
dtype: int64

In [7]:
train.isnull().sum()

text         0
sentiment    0
dtype: int64

In [8]:
train['sentiment'].value_counts()

sentiment
1    90766
2    49798
0    49327
Name: count, dtype: int64

Видим - с данными все в порядке, пропусков нет. Отметим несбалансированность положительных 
примеров.

## Предобработка

Функции предобработки для текста и датасета

In [9]:
morph = MorphAnalyzer()
# nltk.download('stopwords') # Не надо выполнять если пакет уже загружен
stop_words = set(stopwords.words('russian'))


def preprocess_text(text: str) -> list[str]:
    """
    Предобработка текста для сентимент-анализа: удаление стоп-слов и лишних символов, 
    лемматизация, токенизация
    :param text: Текст, который надо обработать
    :return: Список обработанных токенов
    """
    
    text = text.lower()
    text = re.sub(r'[^а-яёЁ?!-+0-9\s]', '', text)
    text = text.replace('!', ' ! ').replace('!?', ' !? ')
    text = re.sub(r'[()\[\]{}]', '', text)

    tokens = text.split()
    tokens = [morph.parse(token)[0].normal_form for token in tokens if token not in stop_words]
    return tokens


def preprocess(dataset: pd.DataFrame) -> None:
    """
    Создание дополнительного признака в датасете - результат применения preprocess_text к тексту.
    :param dataset: Датасет, который надо обработать
    :return: None
    """

    tqdm.pandas(desc="Обработка датасета")

    dataset["prep_text"] = dataset["text"].progress_apply(preprocess_text)
    
    return None

Применение функции предобработки к датасетам и их сохранение в зависимости от того, было ли это 
сделано ранее.  

In [10]:
prep_train_file = os.path.join(data_dir, 'prep_train.csv')
prep_test_file = os.path.join(data_dir, 'prep_test.csv')

if not os.path.exists(prep_train_file):
    print("Обработка тренировочного датасета")
    preprocess(train)
    train.to_csv(prep_train_file, index=False)  # Сохранение обработанных данных
    print(f"Обработанные тренировочные данные сохранены")


if not os.path.exists(prep_test_file):
    print("Обработка тестового датасета")
    preprocess(test)
    test.to_csv(prep_test_file, index=False)  # Сохранение обработанных данных
    print(f"Обработанные тестовые данные сохранены")

prep_train = pd.read_csv(os.path.join(data_dir, 'prep_train.csv'))
prep_test = pd.read_csv(os.path.join(data_dir, 'prep_test.csv'))

## Машинное обучение

In [11]:
x_train = prep_train['prep_text'].values
y_train = prep_train['sentiment'].values
x_test = prep_test['prep_text'].values
y_test = prep_test['sentiment'].values

Вычислим веса классов

In [12]:
array_class_weights = compute_class_weight(class_weight="balanced",
                                     classes=np.array([0, 1, 2]),
                                     y=y_train)
class_weights = dict(zip([0, 1, 2], array_class_weights))

sample_weights = np.array([class_weights[sentiment] for sentiment in y_train]) # Для XGB

Функция для оценки метрик

In [13]:
def evaluate_model(y_test, y_pred) -> pd.DataFrame:
    """
    Оценка метрик accuracy, precision, recall, f1-score на каждом классе с последующим усреднением
    :param y_test: тестовые таргеты
    :param y_pred: предсказанные таргеты
    :return: 
    """
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    
    metrics = {
        'Метрика': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
        'Значение': [accuracy, precision, recall, f1]
    }
    
    df_metrics = pd.DataFrame(metrics)
    
    return df_metrics

### Тестирование различных моделей c BOW и tf-idf

В качестве векторизаторов протестируем bag of words и tf-idf  

Примечание. Помимо своих Были протестированы встроенные tf-idf и bag of words - они показали себя
 +- также (на 1 тысячную хуже!) на logreg, так что я решил оставить свои векторизаторы

In [14]:
tf_idf = TfidfVectorizer(min_df=0.01, max_df=0.95, max_features=30000, ngram_range=(1, 2))
tf_idf.fit(x_train)
x_train_tf_idf = tf_idf.transform(x_train)
x_test_tf_idf = tf_idf.transform(x_test)

In [15]:
bow = CountVectorizer(min_df=0.01, max_df=0.95, max_features=30000, ngram_range=(1, 2))
bow.fit(x_train)
x_train_bow = bow.transform(x_train)
x_test_bow = bow.transform(x_test)

#### Логистическая регрессия

Логистическая регрессия с мешком слов  
f1 на тренировочной - 0.685  
f1 на тестовой - 0.69

In [ ]:
logreg_bow = optuna.create_study(study_name="logreg_bow", direction="maximize",
                                 sampler=TPESampler(), pruner=HyperbandPruner())

def objective(trial):
    C = trial.suggest_float('C', 0.0001, 15)
    # Закомментировал т.к. multinomial лучше, и если установить параметр multi_class то выскакивает warning ¯\_(•_•)_/¯
    # multi_class = trial.suggest_categorical('multi_class', ['multinomial', 'ovr'])
    
    # model = LogisticRegression(C=C, multi_class=multi_class, max_iter=5000)
    model = LogisticRegression(C=C, max_iter=5000, class_weight=class_weights)
    
    f1_scores = cross_val_score(model, x_train_bow, y_train, cv=5, scoring='f1_macro')
    
    return f1_scores.mean()

logreg_bow.optimize(objective, n_trials=10)

In [ ]:
logreg_bow_model = LogisticRegression(**logreg_bow.best_params, max_iter=5000, class_weight=class_weights)
logreg_bow_model.fit(x_train_bow, y_train)
evaluate_model(y_test, logreg_bow_model.predict(x_test_bow))

Логистическая регрессия с tf-idf  
f1 на тренировочной - 0.69  
f1 на тестовой - 0.694

In [ ]:
logreg_tf_idf = optuna.create_study(study_name="logreg_tf_idf", direction="maximize",
                                    sampler=TPESampler(), pruner=HyperbandPruner())

def objective(trial):
    C = trial.suggest_float('C', 0.0001, 15)
    # multi_class = trial.suggest_categorical('multi_class', ['multinomial', 'ovr'])
    
    # model = LogisticRegression(C=C, multi_class=multi_class, max_iter=2000)
    model = LogisticRegression(C=C, max_iter=5000, class_weight=class_weights)
    
    f1_scores = cross_val_score(model, x_train_tf_idf, y_train, cv=5, scoring='f1_macro')
    
    return f1_scores.mean()

logreg_tf_idf.optimize(objective, n_trials=10)

In [ ]:
logreg_tf_idf_model = LogisticRegression(**logreg_tf_idf.best_params, max_iter=5000, class_weight=class_weights)
logreg_tf_idf_model.fit(x_train_tf_idf, y_train)
evaluate_model(y_test, logreg_tf_idf_model.predict(x_test_tf_idf))

#### Наивный байесовский классификатор

NB с мешком слов  
f1 на тренировочной - 0.603  
f1 на тестовой - 0.609  
Не стоит использовать

In [ ]:
nb_bow = optuna.create_study(study_name="nb_bow", direction="maximize",
                             sampler=TPESampler(), pruner=HyperbandPruner())

def objective(trial):
    alpha = trial.suggest_float('alpha', 0.0001, 10)

    model = MultinomialNB(alpha=alpha, fit_prior=False)

    f1_scores = cross_val_score(model, x_train_bow, y_train, cv=5, scoring='f1_macro')

    return f1_scores.mean()

nb_bow.optimize(objective, n_trials=10)

In [39]:
nb_bow_model = MultinomialNB(**nb_bow.best_params, fit_prior=False)
nb_bow_model.fit(x_train_bow, y_train)
evaluate_model(y_test, nb_bow_model.predict(x_test_bow))

,Метрика,Значение
0,Accuracy,0.608304
1,Precision,0.620620
2,Recall,0.639119
3,F1 Score,0.608729


NB с tf-idf  
f1 на тренировочной - 0.6  
f1 на тестовой - 0.625  
Не стоит использовать

In [ ]:
nb_tf_idf = optuna.create_study(study_name="nb_tf_idf", direction="maximize",
                                sampler=TPESampler(), pruner=HyperbandPruner())

def objective(trial):
    alpha = trial.suggest_float('alpha', 0.0001, 10)

    model = MultinomialNB(alpha=alpha, fit_prior=False)

    f1_scores = cross_val_score(model, x_train_tf_idf, y_train, cv=5, scoring='f1_macro')

    return f1_scores.mean()

nb_tf_idf.optimize(objective, n_trials=10)

In [35]:
nb_tf_idf_model = MultinomialNB(**nb_tf_idf.best_params, fit_prior=False)
nb_tf_idf_model.fit(x_train_tf_idf, y_train)
evaluate_model(y_test, nb_tf_idf_model.predict(x_test_tf_idf))

,Метрика,Значение
0,Accuracy,0.629254
1,Precision,0.626967
2,Recall,0.631249
3,F1 Score,0.624805


#### Метод K-ближайших соседей

KNN с мешком слов - f1 на тренировочной около 0.635 + слишком долго работает - лучше не использовать

In [ ]:
knn_bow = optuna.create_study(study_name="knn_bow", direction="maximize",
                              sampler=TPESampler(), pruner=HyperbandPruner())

def objective(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 1, 29, step=2)

    model = KNeighborsClassifier(n_neighbors=n_neighbors, metric='cosine', n_jobs=4)

    f1_scores = cross_val_score(model, x_train_bow, y_train, cv=5, scoring='f1_macro')

    return f1_scores.mean()

knn_bow.optimize(objective, n_trials=5)

KNN с tf-idf - f1 на тренировочной около 0.637 + слишком долго работает - лучше не использовать

In [ ]:
knn_tf_idf = optuna.create_study(study_name="knn_tf_idf", direction="maximize",
                                 sampler=TPESampler(), pruner=HyperbandPruner())

def objective(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 1, 29, step=2)

    model = KNeighborsClassifier(n_neighbors=n_neighbors, metric='cosine', n_jobs=4)

    f1_scores = cross_val_score(model, x_train_tf_idf, y_train, cv=5, scoring='f1_macro')

    return f1_scores.mean()

knn_tf_idf.optimize(objective, n_trials=5)

#### Решающее дерево

Решающее дерево с мешком слов - f1 на тренировочной около 0.6 + долго обучается - не стоит использовать

In [ ]:
tree_bow = optuna.create_study(study_name="tree_bow", direction="maximize",
                               sampler=TPESampler(), pruner=HyperbandPruner())

def objective(trial):
    max_depth = trial.suggest_int("max_depth", 2, 20)
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])

    model = DecisionTreeClassifier(max_depth=max_depth, criterion=criterion, class_weight=class_weights)

    f1_scores = cross_val_score(model, x_train_bow, y_train, cv=5, scoring='f1_macro')

    return f1_scores.mean()

tree_bow.optimize(objective, n_trials=10)

Решающее дерево с tf-idf  

f1 на тренировочной - 0.6  
f1 на тестовой - 0.59  
Тоже не стоит использовать

In [ ]:
tree_tf_idf = optuna.create_study(study_name="tree_tf_idf", direction="maximize",
                                  sampler=TPESampler(), pruner=HyperbandPruner())

def objective(trial):
    max_depth = trial.suggest_int("max_depth", 2, 20)
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])

    model = DecisionTreeClassifier(max_depth=max_depth, criterion=criterion, class_weight=class_weights)

    f1_scores = cross_val_score(model, x_train_tf_idf, y_train, cv=5, scoring='f1_macro')

    return f1_scores.mean()

tree_tf_idf.optimize(objective, n_trials=10)

In [41]:
tree_tf_idf_model = DecisionTreeClassifier(**tree_tf_idf.best_params)
tree_tf_idf_model.fit(x_train_tf_idf, y_train)
evaluate_model(y_test, tree_tf_idf_model.predict(x_test_tf_idf))

,Метрика,Значение
0,Accuracy,0.611006
1,Precision,0.617303
2,Recall,0.587504
3,F1 Score,0.592788


#### Случайный лес

Случайный лес с мешком слов  
f1 на тренировочной 0.6  
f1 на тестовой - 0.575  
Не стоит использовать

In [ ]:
rnd_forest_bow = optuna.create_study(study_name="rnd_forest_bow", direction="maximize",
                                     sampler=TPESampler(), pruner=HyperbandPruner())

def objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 50, 500)
    max_depth = trial.suggest_int("max_depth", 2, 20)

    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, 
                                   class_weight=class_weights, n_jobs=4)
    
    f1_scores = cross_val_score(model, x_train_bow, y_train, cv=5, scoring='f1_macro')
    
    return f1_scores.mean()

rnd_forest_bow.optimize(objective, n_trials=10)

In [17]:
rnd_forest_bow_model = RandomForestClassifier(**rnd_forest_bow.best_params, n_jobs=4)
rnd_forest_bow_model.fit(x_train_bow, y_train)
evaluate_model(y_test, rnd_forest_bow_model.predict(x_test_bow))

,Метрика,Значение
0,Accuracy,0.638070
1,Precision,0.698947
2,Recall,0.561610
3,F1 Score,0.575016


Случайный лес с tf-idf  
f1 на тренировочной - около 0.667, но обучается очень долго

In [ ]:
rnd_forest_tf_idf = optuna.create_study(study_name="rnd_forest_tf_idf", direction="maximize",
                                        sampler=TPESampler(), pruner=HyperbandPruner())

def objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 50, 500)
    max_depth = trial.suggest_int("max_depth", 2, 30)

    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, 
                                   class_weight=class_weights)
    
    f1_scores = cross_val_score(model, x_train_tf_idf, y_train, cv=5, scoring='f1_macro')
    
    return f1_scores.mean()

rnd_forest_tf_idf.optimize(objective, n_trials=10)

In [ ]:
rnd_forest_tf_idf_model = RandomForestClassifier(**rnd_forest_tf_idf.best_params, n_jobs=4)
rnd_forest_tf_idf_model.fit(x_train_tf_idf, y_train)
evaluate_model(y_test, rnd_forest_tf_idf_model.predict(x_test_tf_idf))

#### Градиентный бустинг

Градиентный бустинг с мешком слов  
f1 на тренировочной - 0.719
f1 на тестовой - 0.72

In [ ]:
xgb_bow = optuna.create_study(study_name="xgb_bow", direction="maximize",
                              sampler=TPESampler(), pruner=HyperbandPruner())

def objective(trial):
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.9)
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    max_depth = trial.suggest_int('max_depth', 3, 15)
    
    model = XGBClassifier(learning_rate=learning_rate, n_estimators=n_estimators, 
                          max_depth=max_depth, n_jobs=4)
    
    f1_scores = cross_val_score(model, x_train_bow, y_train, cv=5, scoring='f1_macro',
                                params={'sample_weight': sample_weights})
    
    return f1_scores.mean()

xgb_bow.optimize(objective, n_trials=10)

In [18]:
# Взяты из логов optuna для лучшей попытки, просто перезагружал jupyter
best_params = {'learning_rate': 0.1506113744991435, 'n_estimators': 337, 'max_depth': 14}

xgb_bow_model = XGBClassifier(**best_params, n_jobs=4)
xgb_bow_model.fit(x_train_bow, y_train)
evaluate_model(y_test, xgb_bow_model.predict(x_test_bow))

,Метрика,Значение
0,Accuracy,0.733055
1,Precision,0.724450
2,Recall,0.718184
3,F1 Score,0.720614


Градиентный бустинг с tf-idf  
f1 на тренировочной - чуть хуже, чем с BOW + обучается в разы дольше

In [ ]:
xgb_tf_idf = optuna.create_study(study_name="xgb_tf_idf", direction="maximize",
                                 sampler=TPESampler(), pruner=HyperbandPruner())

def objective(trial):
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.9)
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    max_depth = trial.suggest_int('max_depth', 2, 10)
    
    model = XGBClassifier(learning_rate=learning_rate, n_estimators=n_estimators, 
                          max_depth=max_depth, n_jobs=4)
    
    f1_scores = cross_val_score(model, x_train_tf_idf, y_train, cv=5, scoring='f1_macro',
                                params={'sample_weight': sample_weights})
    
    return f1_scores.mean()

xgb_tf_idf.optimize(objective, n_trials=10)

**Как видно, лучший (но все еще плохой) результат на тестовой выборке (f1 = 0.72) показал градиентный бустинг с BOW**  
Сохраним модель ()

In [20]:
dump(xgb_bow_model, project_dir + '\models\\xgb_bow_model.joblib')

['C:\\programming\\Projects\\REU_Data_Science_Club\\Sentiment_Analysis_SMM\\models\\xgb_bow_model.joblib']

Судя по всему, bag of words и tf-idf очень плохо подходят для описания имеющихся текстов  
Стоит попробовать word embeddings (смотреть в другом ноутбуке)

### Тестирование различных моделей с word2vec

Обучение word2vec в зависимости от того, есть ли уже готовый в папке models, и сохранение

In [21]:
word2vec_path = project_dir + "\models\\word2vec.model"

if not os.path.exists(word2vec_path):
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', 
                    level=logging.INFO)
    
    word2vec = Word2Vec(sentences=x_train,
                        vector_size=150,   # Размер векторов
                        window=7,          # Размер окна для контекстных слов
                        min_count=10,      # Минимальная частота слова для того, чтобы учесть его
                        sg=1,              # skip-gram вместо CBOW
                        workers=4,         # Ядра процессора
                        epochs=10)         # Кол-во эпох
    
    word2vec.save(word2vec_path)
    
    logging.basicConfig(level=logging.WARNING)

else:
    word2vec = Word2Vec.load(word2vec_path)

Напишем функцию для векторизации списка токенов

In [22]:
def word2vec_vectorize(text):
    """
    Создание среднего вектора из векторов слов (которые есть в словаре) в токенезированном тексте
    :param text: Текст - список токенов
    :return: Векторное представление текста 
    """
    
    words_vecs = [word2vec.wv[word] for word in text if word in word2vec.wv]
    if not len(words_vecs):
        return np.zeros(word2vec.vector_size)
    words_vecs = np.array(words_vecs)
    
    return words_vecs.mean(axis=0)

Применение функции к тренировочной и тестовой выборкам в зависимости от того, есть ли сохраненные 
векторы в папке data, и сохранение получившихся векторов

In [31]:
x_train_word2vec_path = project_dir + '\data\\x_train_word2vec.npz'
x_test_word2vec_path = project_dir + '\data\\x_test_word2vec.npz'

if not (os.path.exists(x_train_word2vec_path) and os.path.exists(x_test_word2vec_path)):
    x_train_word2vec = np.array([word2vec_vectorize(text) for text in tqdm(x_train)])
    x_train_word2vec = x_train_word2vec.astype(np.float32)
    np.savez_compressed(x_train_word2vec_path, x_train_word2vec)
    
    x_test_word2vec = np.array([word2vec_vectorize(text) for text in tqdm(x_test)])
    np.savez_compressed(x_train_word2vec_path, x_train_word2vec)
    x_test_word2vec = x_test_word2vec.astype(np.float32)

else:
    x_train_word2vec = np.load(x_train_word2vec_path)
    x_test_word2vec = np.load(x_test_word2vec_path)

#### Логистическая регрессия

Работает плохо с word2vec (f1 = 0.5)

In [ ]:
logreg_word2vec = optuna.create_study(study_name="logreg_word2vec", direction="maximize",
                                 sampler=TPESampler(), pruner=HyperbandPruner())

def objective(trial):
    C = trial.suggest_float('C', 0.0001, 10)

    model = LogisticRegression(C=C, max_iter=1000)
    
    f1_scores = cross_val_score(model, x_train_word2vec, y_train, cv=5, scoring='f1_macro', 
                                class_weight=class_weights)
    
    return f1_scores.mean()


logreg_word2vec.optimize(objective, n_trials=5)

#### Градиентный бустинг

In [ ]:
xgb_word2vec = optuna.create_study(study_name="xgb_word2vec", direction="maximize",
                              sampler=TPESampler(), pruner=HyperbandPruner())

def objective(trial):
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.9)
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    
    model = XGBClassifier(learning_rate=learning_rate, n_estimators=n_estimators, 
                          max_depth=max_depth, n_jobs=4)
    
    f1_scores = cross_val_score(model, x_train_word2vec, y_train, cv=5, scoring='f1_macro',
                                params={'sample_weight': sample_weights})
    
    return f1_scores.mean()

xgb_word2vec.optimize(objective, n_trials=10)

**Вывод. Пока что наилучший результат дает XGB с tf-idf. Думаю, что стоит попробовать другие 
эмбеддинги.**